In [1]:
import torch
import datawrangler as dw

2023-08-10 12:49:27.880353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
import json

In [3]:
filepath = "/Users/f004p74/Desktop/deathblart-nlp/outputs/"
json_files = ["Blart2015","Blart2016","Blart2017","Blart2018","Blart2019","Blart2020","Blart2021","Blart2022"]

bert = {'model': 'TransformerDocumentEmbeddings', 'args': ['bert-base-uncased'], 'kwargs': {}}

In [4]:
json_dicts = []

for file in json_files:
    with open(filepath+file+".json","r") as read_file:
        whisper_data = json.load(read_file)
    json_dicts.append(whisper_data)

In [8]:
for episode,whisper_data in enumerate(json_dicts):
    whisper_segments, start_times, end_times = [], [], [] 
    
    for segment in range(len(whisper_data)):
        whisper_segments.append(whisper_data[segment]["text"])
        start_times.append(whisper_data[segment]["start"])
        end_times.append(whisper_data[segment]["end"])
        
    embeddings = pd.DataFrame()
    
    for segment in whisper_segments:
        bert_embeddings = dw.wrangle(segment, text_kwargs={'model': bert})
        embeddings = embeddings.append(bert_embeddings, ignore_index = True)
    
    embeddings["start"] = start_times
    embeddings["end"] = end_times
    embeddings["episode"] = json_files[episode]
    
    embeddings.to_csv(json_files[episode]+"_embeddings.csv", index=False)